*********************************************************************************************************
# A Tour of Python 3  
version 1.0.1  
Authors: Phil Pfeiffer, Zack Bunch, and Feyisayo Oyeniyi  
East Tennessee State University  
Last updated June 2021  
*********************************************************************************************************

# 13. Packages   
 13.1 [Overview](#Packages-Overview)  
 13.2 [Using `__init__` to Configure Package Imports](#Packages-Using-__init__)

## 13.1 Overview <a name='Packages-Overview'></a>

A *package* is a directory that contains Python files. Its name must conform to rules for naming Python identifiers: for example, 'myPackage' and 'my_package' are valid names for package directories;'my Package' and 'my-Package' are not.

Python packages are accessed using `import` statements. In order for a Python code to access a package, the directory that contains that package must be named in `sys.path`. If, for example, a directory on `sys.path` contains a package directory name *pkg*, then *pkg*'s contents can be loaded using the statement `import pkg`. 

A package directory may hold sub-directories and files. Continuing the previous example, if *pkg* contains a package directory, *subpkg*, then *subpkg*'s contents can be accessed using the statement `import pkg.subpkg`. These conventions should be familiar to anyone who's worked with packages in the Java programming language.

Note:  The `importlib.reload` commands override Python's suppression of attempts to import a module that's already been imported. This caching of previously imported modules optimizes execution by avoiding the useless reimportation of code that hasn't changed. It also, however, creates a need to force the reloading of modules that were updated during a Python session, like the modules created and recreated by these examples.

In [ ]:
# 13.1  Creating, then accessing, a package.

# supporting constants
CREATE_AS_NEW_FILE = 'x'    # file access mode for open()

# library resources
import os, sys, importlib, shutil

# supporting functions
make_printable = lambda exception: '' if str(exception) is None else str(exception)

# precondition check: ensure that we're creating a totally new, unreferenced module in a totally new directory

package_name = 'my_package'
corrective_action = f'please ensure that {package_name} names a non-existent file system object'
assert not os.path.exists( package_name ), corrective_action

package_module_name = 'my_module'
package_module_path= f'{package_name}/{package_module_name}.py'
module_fn = 'sample_function'

importlib.invalidate_caches()   # clear stale module data, possibly from previous exercise

# preconditions established: create a package in the local directory, then try to import and use it

try:
  #  create the package directory
  #
  os.mkdir( package_name )
  try:
    return_message = f'{module_fn} in {package_module_path}: {{x}}+{{y}} is {{x+y}}'
    function_header  = f'def {module_fn}(x, y):'
    function_body    = "return f'" + return_message + "'"
    try:
      #  create the function's lone module
      #  
      package_module_fd = open( package_module_path, CREATE_AS_NEW_FILE )
      package_module_fd.write( function_header + '\n' )
      package_module_fd.write( '  ' + function_body + '\n' )
      package_module_fd.close()
      try:
        #  (re)import the module and execute the function
        #  
        exec( f'import {package_name}' )
        exec( f'importlib.reload( {package_name} )' )
        exec( f'import {package_name}.{package_module_name}' )
        exec( f'importlib.reload( {package_name}.{package_module_name} )' )
        try:
          result_from_sample_function = eval( f'{package_name}.{package_module_name}.{module_fn}(3, 4)' )
          print( result_from_sample_function )
        except Exception as exception:
          print( f"can't access {package_name}.{package_module_name}.{module_fn}: {make_printable(exception)}" )
      except Exception as exception:
        print( f"can't import {package_name}: {make_printable(exception)}" )
    except Exception as exception:
      print( f"can't create file ({package_module_path}): {make_printable(exception)}" )
  except Exception as exception:
    print( f"can't create file ({package_init_path}): {make_printable(exception)}" )
  finally:
    try:
      shutil.rmtree( package_name )
    except Exception as exception:
      print( f"can't remove directory ({package_name}): {make_printable(exception)}" )
except Exception as exception:
  print( f"can't create directory ({package_name}): {make_printable(exception)}" )

## 13.2 Using `__init__` to Configure Package Imports <a name='Packages-Using-__init__'></a>

Before Python 3.3, packages were required to include a file named `__init__.py` in their home directories. While this requirement has been lifted, including `__init__.py` can be useful for configuring package operation. When a package is first imported, Python executes code in `__init__.py`. Accordingly, one use of `__init__.py` is to define initialize package variables.  

This example also illustrates the use of relative path expressions to import values. Note that the sample function must import its package's variable in order to access it.

To see how the package is structured and what it contains, comment out the call to `shutil.rmtree` in the example's `finally` block and replace the statement with a `pass` statement.

In [ ]:
# 13.2  Creating, then accessing, a package that uses `__init__.py` to predefine a variable.

# supporting constants
CREATE_AS_NEW_FILE = 'x'    # file access mode for open()

# library resources
import os, sys, importlib, shutil

# supporting functions
make_printable = lambda exception: '' if str(exception) is None else str(exception)

# precondition check: ensure that we're creating a totally new, unreferenced module in a totally new directory

package_name = 'my_package'
corrective_action = f'please ensure that {package_name} names a non-existent file system object'
assert not os.path.exists( package_name ), corrective_action

package_init_name = '__init__'
package_init_path= f'{package_name}/{package_init_name}.py'
package_module_name = 'my_module'
package_module_path= f'{package_name}/{package_module_name}.py'
package_var = 'a'
module_fn = 'sample_function'

importlib.invalidate_caches()   # clear stale module data, possibly from previous exercise

# preconditions established: create a package in the local directory, then try to import and use it

try:
  #  create the package directory
  #
  os.mkdir( package_name )
  try:
    #  create and configure the __init__ module
    #  
    package_init_fd = open( package_init_path, CREATE_AS_NEW_FILE )
    package_init_fd.write( package_var + ' = 3' + '\n' )
    package_init_fd.close()
    #  define the package's lone function
    #  
    function_header  = f'def {module_fn}( ):'
    return_message = f'{module_fn} in {package_module_path}: {package_var} is '
    function_body = "return f'" + return_message + "'" + ' + ' + 'str(' + f'{package_var}' + ')'
    try:
      #  create the function's lone module
      #  
      package_module_fd = open( package_module_path, CREATE_AS_NEW_FILE )
      package_module_fd.write( function_header + '\n' )
      package_module_fd.write( '  ' + 'from . import ' + package_var + '\n' )
      package_module_fd.write( '  ' )
      package_module_fd.write( function_body )
      package_module_fd.write( '\n' )
      package_module_fd.close()
      try:
        #  (re)import the module and execute the function
        #  
        exec( f'import {package_name}' )
        exec( f'importlib.reload( {package_name} )' )
        exec( f'import {package_name}.{package_module_name}' )
        exec( f'importlib.reload( {package_name}.{package_module_name} )' )
        try:
          result_from_sample_function = eval( f'{package_name}.{package_module_name}.{module_fn}( )' )
          print( result_from_sample_function )
        except Exception as exception:
          print( f"can't access {package_name}.{package_module_name}.{module_fn}: {make_printable(exception)}" )
      except Exception as exception:
        print( f"can't import {package_name}: {make_printable(exception)}" )
    except Exception as exception:
      print( f"can't create file ({package_module_path}): {make_printable(exception)}" )
  except Exception as exception:
    print( f"can't create file ({package_init_path}): {make_printable(exception)}" )
  finally:
    try:
      shutil.rmtree( package_name )
    except Exception as exception:
      print( f"can't remove directory ({package_name}): {make_printable(exception)}" )
except Exception as exception:
  print( f"can't create directory ({package_name}): {make_printable(exception)}" )

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 13.2.1:**

</span><span style='color:navy'>In the following code cell, modify the previous example so that `sample_function` takes one parameter, an int, and returns the sum of this parameter and `a`. Illustrate the code's operation by invoking it at least twice with nonzero arguments and printing the return values.</span>